This file acts like my draft papers for developing the solver. Once I figure out something, the code will be put into production in the .py files


In [3]:
import numpy as np
import torch
SEED= 9999
np.random.seed(SEED)
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import seaborn as sns
ONE_KB = 1000

In [6]:
A_fn = '/gladstone/engelhardt/home/hvu/source/RNA_rates/splicingrates/simulations/elong_analysis/A0.33_0.2.pt'
A = torch.load(A_fn)  # (n, m)
b = torch.ones((A.shape[0], 1)) * 5 # (n, 1)
coordinates = 0.2* torch.arange(A.shape[1])+0.1 # (m, 1)

/tmp/ipykernel_2726585/3681833607.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  A = torch.load(A_fn)  # (n, m)


In [11]:
import torch
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import LinearMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.kernels import RBFKernel

class CustomKernel(gpytorch.kernels.Kernel):
    def __init__(self, coordinates, X):
        super().__init__()
        self.coordinates = coordinates
        self.X = X  # (n, m) where n: # samples, m: # features
        self.base_kernel = RBFKernel()
    def forward(self):
        # Compute RBF kernel for coordinates
        coord_covar = self.base_kernel(self.coordinates, self.coordinates)
        # Compute X^T * RBF(coordinates) * X
        return self.X @ coord_covar @ self.X.t()

class BayesianLinearRegressionGP(ExactGP):
    def __init__(self, train_x, train_y, coordinates, mean_prior, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = LinearMean(input_size=train_x.size(-1))
        self.covar_module = CustomKernel(coordinates, train_x)
        self.mean_module.weights.data = mean_prior
        self.mean_module.bias.data.fill_(0.0)
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x, x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
# Assuming X, Y, coordinates, and mean_prior are your data
X = torch.tensor(A)
y = torch.tensor(b)
coordinates = torch.tensor(coordinates)
mean_prior = 0.33 * torch.ones(coordinates.shape)  # Prior mean is 0
mean_prior = torch.tensor(mean_prior)

likelihood = GaussianLikelihood()
model = BayesianLinearRegressionGP(X, y, coordinates, mean_prior, likelihood)

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(100):
    optimizer.zero_grad()
    output = model(X)
    loss = -mll(output, y)
    loss.backward()
    optimizer.step()

# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

/tmp/ipykernel_2726585/1042685630.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(b)
/tmp/ipykernel_2726585/1042685630.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coordinates = torch.tensor(coordinates)
/tmp/ipykernel_2726585/1042685630.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean_prior = torch.tensor(mean_prior)


RuntimeError: expected scalar type Double but found Float

In [ ]:
# eventually, I want to draw a heatmap of A